In [ ]:
# libraries
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import matplotlib.pyplot as plt
from time import sleep
import inspect
import os
from bs4 import BeautifulSoup
import requests

# Get the search term and tracking period from the user
search_term = input("Please enter the name of the product you want to search: ")
months =input("Please enter the number of months you want to track the product: ")

# To ensure that the user enters a non-string value 
while not months.isdigit():
    print("Warning: Please enter a valid integer value for the number of months.")
    months = input("Please enter the number of months you want to track the product: ")
months = int(months)

# Start the web driver and go to the Hepsiburada homepage
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)


module_path="C:/Users/murat/Desktop/hepsiburada_price_tracker/chromedriver.exe"

driver = webdriver.Chrome(executable_path=module_path,options=chrome_options)
driver.maximize_window()
driver.get("https://www.hepsiburada.com/")

# Accept cookies
driver.find_element_by_id('onetrust-accept-btn-handler').click()
sleep(3)



# Enter the search term in the search box and press Enter
wait = WebDriverWait(driver, 15)
search_box = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'theme-IYtZzqYPto8PhOx3ku3c')))

search_box.send_keys(search_term)
search_box.send_keys(Keys.RETURN)

# Wait for search results and select the first product
sleep(3)  # Sayfanın yüklenmesi için birkaç saniye bekleyin

# Click on the order button
driver.find_element_by_class_name('horizontalSortingBar-Ce404X9mUYVCRa5bjV4D').click()
sleep(3)

# Sort by increasing price
driver.find_element_by_class_name('horizontalSortingBar-PkoDOH7UsCwBrQaQx9bn').click()
sleep(3)

# Get the link, name, and price of the first product in the search results
results = driver.find_elements_by_xpath("//h3[@data-test-id='product-card-name']")
if not results:
    print("Sorry, we could not find the product you were looking for.")
    #driver.quit()
else:
    first_result0 = results[0]
    first_result= first_result0.text
    print(first_result)
    product_link = first_result.find_element(By.XPATH, ".//a[@data-productid]")
    product_url = product_link.get_attribute("href")
    product_name = first_result.find_element(By.XPATH, ".//h3").text
    print("The product selected from the search results is {}: {}".format(product_name, product_url))




In [ ]:
# Seçilen ürün sayfasına gidin
driver.get(product_url)

# Fiyatları alın
prices = []
dates = []
while len(prices) < months:
    price_elems = driver.find_elements(By.XPATH, "//div[@class='price-area']//strong[@itemprop='price']")
    date_elems = driver.find_elements(By.XPATH, "//div[@class='product-info']//span[@class='product-info-date']")
    
    for price_elem, date_elem in zip(price_elems, date_elems):
        price = float(price_elem.text.replace('.', '').replace(',', '.'))
        date = pd.to_datetime(date_elem.text, format='%d %B %Y')
        prices.append(price)
        dates.append(date)
        
    next_button = driver.find_element(By.XPATH, "//a[@class='page-next']")
    if 'disabled' in next_button.get_attribute('class'):
        break
    else:
        driver.execute_script("arguments[0].click();", next_button)

In [18]:
# DataFrame oluşturun ve son üç ayın verilerini seçin
df = pd.DataFrame({'Tarih': dates, 'Fiyat': prices})
start_date = pd.Timestamp.today() - pd.DateOffset(months=months)
end_date = pd.Timestamp.today()
df = df.loc[(df['Tarih'] >= start_date) & (df['Tarih'] <= end_date)]

# Grafik oluşturun
plt.plot(df['Tarih'], df['Fiyat'])
plt.title('Son {} Ayın {} Fiyat Takibi'.format(months, product_name))

NameError: name 'dates' is not defined

In [ ]:
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import matplotlib.pyplot as plt
from time import sleep
import inspect
import os
from bs4 import BeautifulSoup
import requests


# Get the search term and tracking period from the user
search_term = input("Please enter the name of the product you want to search: ")
months =input("Please enter the number of months you want to track the product: ")

# To ensure that the user enters a non-string value 
while not months.isdigit():
    print("Warning: Please enter a valid integer value for the number of months.")
    months = input("Please enter the number of months you want to track the product: ")
months = int(months)


# Start the web driver and go to the Hepsiburada homepage
options = uc.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs",prefs)
driver = uc.Chrome(options=options)

url = 'https://www.hepsiburada.com/'
driver.get(url)
wait = WebDriverWait(driver, 15)

# close cookies bar
wait.until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))).click()

# Enter the search term in the search box and press Enter
search_box = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'theme-IYtZzqYPto8PhOx3ku3c')))
search_box.send_keys(search_term + Keys.RETURN)

# Click on the order button and sort by increasing price
wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'horizontalSortingBar-Ce404X9mUYVCRa5bjV4D'))).click()
wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'horizontalSortingBar-PkoDOH7UsCwBrQaQx9bn'))).click()


# load all products
number_of_products = int(wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'searchResultSummaryBar-AVnHBWRNB0_veFy34hco')))[1].text)
### visibility_of_all_elements_located is a wait strategy in Selenium that checks if all elements of a certain type are visible on the page and waits until they become visible before continuing.


number_of_loaded_products = 0
while number_of_loaded_products < number_of_products:
    
    loaded_products = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'li[class*=productListContent][id]'))) # select all products
    number_of_loaded_products = len(loaded_products)
    print("number_of_loaded_products",number_of_loaded_products)
    driver.execute_script('arguments[0].scrollIntoView({block: "center", behavior: "smooth"});', loaded_products[-1])
    sleep(3)

# Get the link, name, price and seller of all the products
product_names  = [h3.text for h3 in driver.find_elements(By.CSS_SELECTOR, 'h3[data-test-id=product-card-name]')]
product_urls   = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, 'a[class*=ProductCard]')]
product_prices = [div.text for div in driver.find_elements(By.CSS_SELECTOR, 'div[data-test-id=price-current-price]')]
product_sellers = []
if not product_names:
    print("Sorry, we could not find the product you were looking for.")
    driver.quit()
else:
   
    for i,url in enumerate(product_urls[:10]):
        print(f'search seller names {i+1}/{number_of_loaded_products}', end='\r')
        driver.get(url)
        name = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.seller a'))).text
        product_sellers.append(name)

    print(f"Products found: {number_of_loaded_products}     \n"+
          f"The product selected from the search results is:\n"+
          f"name: {product_names[0]}\n"+
          f"price: {product_prices[0]}\n"+
          f"seller: {product_sellers[0]}\n"+
          f"url: {product_urls[0]}")


Please enter the name of the product you want to search:  mama


In [ ]:
number_of_products

In [ ]:
a

In [23]:
number_of_loaded_products


1

In [33]:
google_active_view_elements

[]